In [138]:
# Importing packages
#from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
#import  sys

In [139]:
url = 'https://cenoteka.rs'

In [140]:
# Find all categories
def find_categories():
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    categories = []
    classes = soup.find(class_='nav navbar-nav mr-auto')
    for i in classes.find_all(class_='nav-item dropdown'):
        categories.append(i.find('a')['href'])

    categories = categories[1:]
    return categories


In [141]:
# Find all links in categories
def get_links(category):
    page = requests.get(url+category)
    soup = bs(page.content, 'html.parser')
    links = []
    divs = soup.find_all(class_='cat-list-item')
    for a in divs:
        links.append(a.find('a')['href'])
    return links


In [142]:
def scrape_page(soup):
    
    shops_class = soup.find("div", {"id": "products"})
    #category = soup.find(class_='section-title col-12').get_text()
    #print(shops_class)
    prices_list = []
    names_list = []
    product_dict = {}
    for shop in shops_class.find_all(True, {'class':['border-gray d-flex flex-nowrap flex-row article-row col-6 col-md-12 p-0', 'border-gray d-flex flex-nowrap flex-row article-row col-6 col-md-12 p-0 odd']}):
        names=shop.find_all(class_='article-hover article-name flex-full-height-center border-gray-right p-1')
        #print(names)
        for name in names:
            
            names_list.append(name.get_text())
            names_list = [item.strip() for item in names_list if str(item)]
            #print(names_list)
        prices = shop.find_all(class_='price')
        for price in prices:
            prices_list.append(price.get_text())
            prices_list = [item.strip() for item in prices_list if str(item)]
        
        product_dict[names_list[0]] = prices_list
        prices_list = []
        names_list = []
        
    return product_dict
        

In [145]:
dataframe_full = pd.DataFrame()
categories = find_categories()
#print(categories)
# Iterate through each category and scrape links
for category in categories:
    links = get_links(category)
    
    for link in links:
        #print(link)
        page = requests.get(url+link)
        soup = bs(page.content, 'html.parser')
        try:
            pages = soup.find(class_="paginator-navigation")
            page_num = pages.find_all(class_="page-item")[-2]
            page_num = int(page_num.get_text())
            
            for i in range(1,page_num+1):
                #print(i)
                dataframe = pd.DataFrame()
                page = requests.get(url+link+'?page='+str(i))
                soup = bs(page.content, 'html.parser')
                product = scrape_page(soup)
                #print(product)
                dataframe = dataframe.from_dict(product, orient='index', columns= ['IDEA','MAXI','UNIVEREXPORT','TEMPO','DIS','RODA','LIDL'])
                #print(dataframe)
                dataframe_full = dataframe_full.append(dataframe)

        except:
            page = requests.get(url+link)
            soup = bs(page.content, 'html.parser')
            #print('Exception RAISED')
            #print(soup)
            product = scrape_page(soup)
            try:
                dataframe = dataframe.from_dict(product, orient='index', columns= ['IDEA','MAXI','UNIVEREXPORT','TEMPO','DIS','RODA','LIDL'])
                #print(dataframe)
                dataframe_full = dataframe_full.append(dataframe)
            except:
                pass
        
        


In [146]:
dataframe_full

,IDEA,MAXI,UNIVEREXPORT,TEMPO,DIS,RODA,LIDL
Brašno DANUBIUS heljdin mešani hleb 1kg,"294,99","294,99","224,99","274,99",-,"274,99",-
Brašno DANUBIUS meko T-400 1kg,"84,70","84,70","84,70","84,70","84,69","79,99",-
Brašno DANUBIUS oštro T-400 1kg,"97,99",-,"94,99","92,99","81,99","92,99",-
Brašno DANUBIUS T-500 1kg,"78,65","78,65","78,60","78,65","78,59","78,65",-
Brašno DANUBIUS T-850 900g,"76,99","94,99","69,99","84,99","84,99","76,99",-
...,...,...,...,...,...,...,...
Zamensko mleko NAN Optipro 2 6m 800g,"2.299,99","2.009,00",-,"1.996,99","2.138,99","2.179,99","2.119,00"
Zamensko mleko NAN Optipro 3 12m 800g,-,"1.663,99",-,"1.993,00","2.012,99",-,"2.029,00"
Zamensko mleko NOVALAC 1 400g,-,"929,99",-,"919,99",-,-,"779,00"
Zamensko mleko NOVALAC 2 400g,-,"929,99",-,"919,99",-,-,"799,00"


In [148]:
dataframe_full.to

<class 'pandas.core.frame.DataFrame'>
Index: 3488 entries, Brašno DANUBIUS heljdin mešani hleb 1kg to Zamensko mleko NOVALAC 3 400g
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IDEA          3488 non-null   object
 1   MAXI          3488 non-null   object
 2   UNIVEREXPORT  3488 non-null   object
 3   TEMPO         3488 non-null   object
 4   DIS           3488 non-null   object
 5   RODA          3488 non-null   object
 6   LIDL          3488 non-null   object
dtypes: object(7)
memory usage: 218.0+ KB
